In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils import data

import os
import pandas as pd
import numpy as np
from torchvision import transforms
from collections import OrderedDict
from PIL import Image

/opt/conda/lib/python3.6/site-packages/torchvision/io/_video_opt.py:17: UserWarning: video reader based on ffmpeg c++ ops not available
  warnings.warn("video reader based on ffmpeg c++ ops not available")


In [2]:
from dataloader import RetinopathyLoader

In [3]:
train_set = RetinopathyLoader(root='./data/',mode='train')
train_loader = data.DataLoader(dataset=train_set,batch_size=64,shuffle=True)
test_set = RetinopathyLoader(root='./data/',mode='test')
test_loader = data.DataLoader(dataset=test_set,batch_size=64,shuffle=True)

> Found 28099 images...
> Found 7025 images...


In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)